## Cocktail Title generator
##### By: Mystery Machine
Just enter the names of the ingredients, instructions to prepare and BOOM!
Our model will predict a few unheard-of names for your cocktail!😉


In [4]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Party here!